In [1]:
# imports
import requests
import pandas as pd

### Using the code from the lesson as a guide, create a dataframe named items that has all of the data for items.

In [87]:
# variable holds python object that represents an HTTP response from the given url
response = requests.get('https://python.zach.lol/api/v1/items')

# save contents of response in format we can access more easily
data = response.json()

# save data from items section of data
items = pd.DataFrame(data['payload']['items'])

# print max pages and next page so we know how many pages there are and what the next page is
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 3
next_page: /api/v1/items?page=2


In [88]:
# save url without page specifications
base_url = 'https://python.zach.lol'

# save requests.get function that uses url above concatenated with string
# end result is new url that gets data from next page being passed to requests.get function
response = requests.get(base_url + data['payload']['next_page'])

# save retrieved data
data = response.json()

# print max pages and next page so we know how many pages there are and what the next page is
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

# concat new dataframe with old dataframe and drop index
items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index(drop = True)

# check shape to ensure new DF is growing correctly
items.shape

max_page: 3
next_page: /api/v1/items?page=3


(40, 6)

In [89]:
# concat next page url info with base url and pass to requests.get function
response = requests.get(base_url + data['payload']['next_page'])

# save data in json format
data = response.json()

# print max and next page info
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

# concat new df to old df and drop index
items = pd.concat([items, pd.DataFrame(data['payload']['items'])]).reset_index(drop=True)

# check shape to ensure new DF is growing correctly 
items.shape

max_page: 3
next_page: None


(50, 6)

In [90]:
# preview data
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [100]:
# converting work to function
def get_items_data(base_url):
    '''
    Accepts url. Create df using data from base url 
    then retrieves data from each sequential page and combines all into one DF.
    '''
    # saving object retrieved from requests function being passed argument URL
    response = requests.get(base_url)
    
    # saving data from .json object
    data = response.json()
    
    # saving first page of items data to df
    df = pd.DataFrame(data['payload']['items'])
    
    # iterating range = 1 through max page count
    for i in range (1, data['payload']['max_page']):
        # passing url for next page to request.get function
        response = requests.get(base_url[:23] + data['payload']['next_page'])
        
        # saving data from response in .json format
        data = response.json()
        
        # concating each new page's data to original df
        df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index(drop=True)
    
    # returning df
    return df

In [102]:
# testing function

# saving url to retrieve first page of items data
items_base_url = 'https://python.zach.lol/api/v1/items'

# saving function with base url passed to it
all_items = get_items_data(items_base_url)

# checking shape to ensure all data gathered
all_items.shape

(50, 6)

### Do the same thing, but for stores.

In [48]:
# variable holds python object that represents an HTTP response from the given url
response = requests.get('https://python.zach.lol/api/v1/stores')

# saving data from response in .json format
data = response.json()

# creating df from stores data
stores = pd.DataFrame(data['payload']['stores'])

# checking max page and next page
# surprisingly, theres only one page
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 1
next_page: None


In [49]:
# previewing data
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [107]:
# checking column and row count
stores.shape

(10, 5)

In [104]:
# converting work to function
def get_stores_data(base_url):
    '''
    Accepts url. Create df using data from base url 
    then retrieves data from each sequential page and combines all into one DF.
    '''
    # saving object retrieved from requests function being passed argument URL
    response = requests.get(base_url)
    
    # saving data from .json object
    data = response.json()
    
    # saving first page of stores data to df
    df = pd.DataFrame(data['payload']['stores'])
    
    # iterating range = 1 through max page count
    for i in range (1, data['payload']['max_page']):
        # passing url for next page to request.get function
        response = requests.get(base_url[:23] + data['payload']['next_page'])
        
        # saving data from response in .json format
        data = response.json()
        
        # concating each new page's data to original df
        df = pd.concat([df, pd.DataFrame(data['payload']['stores'])]).reset_index(drop=True)
    
    # returning df
    return df

In [106]:
# testing function

# saving url to retrieve first page of stores data
stores_base_url = 'https://python.zach.lol/api/v1/stores'

# saving function with base url passed to it
all_stores = get_stores_data(stores_base_url)

# checking shape to ensure all data gathered
all_stores.shape

(10, 5)

### Extract the data for sales. There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [110]:
# converting work to function
def get_sales_data(base_url):
    '''
    Accepts url. Create df using data from base url 
    then retrieves data from each sequential page and combines all into one DF.
    '''
    # saving object retrieved from requests function being passed argument URL
    response = requests.get(base_url)
    
    # saving data from .json object
    data = response.json()
    
    # saving first page of sales data to df
    df = pd.DataFrame(data['payload']['sales'])
    
    # iterating range = 1 through max page count
    for i in range (1, data['payload']['max_page']):
        # passing url for next page to request.get function
        response = requests.get(base_url[:23] + data['payload']['next_page'])
        
        # saving data from response in .json format
        data = response.json()
        
        # concating each new page's data to original df
        df = pd.concat([df, pd.DataFrame(data['payload']['sales'])]).reset_index(drop=True)
    
    # returning df
    return df

In [112]:
# testing function

# saving url to retrieve first page of sales data
sales_base_url = 'https://python.zach.lol/api/v1/sales'

# saving df produced from function to variable
sales = get_sales_data(sales_base_url)

# previewing data
sales.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [113]:
# checking shape to see how many rows/columns are in df
sales.shape

(913000, 5)

### Save the data in your files to local csv files so that it will be faster to access in the future.

In [114]:
# saving data to file sales.csv
items.to_csv('items.csv', index=False)
stores.to_csv('stores.csv', index=False)
sales.to_csv('sales.csv', index=False)

In [ ]:
# converting to function

# function converts DFs to csv files
def csv_maker(items, stores, sales):
    """
    Accepts 3 DFs. Converts passed DFs to csv files then returns DFs.
    """
    # converting DFs
    items.to_csv('items.csv', index=False)
    stores.to_csv('stores.csv', index=False)
    sales.to_csv('sales.csv', index=False)

    # returning DFs
    return items, stores, sales
    

### Combine the data from your three separate dataframes into one large dataframe.

In [116]:
# creating new df, complete_df, that will hold all 3 DFs

# merging sales and stores DFs by matching store (sales) and store_id (stores)
complete_df = sales.merge(stores, left_on='store', right_on='store_id')

# checking column count to make sure merge was a success
# both source DFs had 5 columns so combined they should have 10
complete_df.shape

(913000, 10)

In [117]:
# adding items DF to complete_df by matching item (complete_df) with item_id (items)
complete_df = complete_df.merge(items, left_on='item', right_on='item_id')

# checking column count to make sure merge was a success
# complete df had 10 columns while items df had 6 columns so new df should have 16
complete_df.shape

(913000, 16)

In [4]:
# converting to function

# function combines 3 DFs from exercise
def df_combiner(sales, stores, items):
    """
    Combines 3 DFs into 1 DF.
    """
    
    # merging sales and stores DFs by matching store (sales) and store_id (stores)
    complete_df = sales.merge(stores, left_on='store', right_on='store_id')
    
    # adding items DF to complete_df by matching item (complete_df) with item_id (items)
    complete_df = complete_df.merge(items, left_on='item', right_on='item_id')
    
    return complete_df

### Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [2]:
# importing data from csv as dataframe
power = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')

# previewing data
power.head()

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.184,NaN,NaN,NaN
1,2006-01-02,1380.521,NaN,NaN,NaN
2,2006-01-03,1442.533,NaN,NaN,NaN
3,2006-01-04,1457.217,NaN,NaN,NaN
4,2006-01-05,1477.131,NaN,NaN,NaN


In [ ]:
# converting to function

# function to acquire german power systems data
def get_power():
    """
    No argument needed. Run function to acquire german power systems data.
    """
    # saving data to variable
    power = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
    # returning df
    return power

### Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

In [5]:
print('All solution converted to functions and added to acquire.py file')

All solution converted to functions and added to acquire.py file
